In [1]:
# Lab 11 MNIST and Convolutional Neural Network
import tensorflow as tf
import random
import time
# import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# hyper parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)#dropout

# L1 ImgIn shape=(?, 28, 28, 1) 1개에서 32개로 conv
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L1=tf.nn.dropout(L1,keep_prob=keep_prob)
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
'''
# L1 ImgIn shape=(?, 28, 28, 1) 1개에서 32개로 conv
W11 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L11 = tf.nn.conv2d(L1, W11, strides=[1, 1, 1, 1], padding='SAME')
L11 = tf.nn.relu(L11)
L11 = tf.nn.max_pool(L11, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L11=tf.nn.dropout(L11,keep_prob=keep_prob)
# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L11, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2=tf.nn.dropout(L2,keep_prob=keep_prob)
L2_flat = tf.reshape(L2, [-1, 4*4* 128])
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
'''

# Final FC 7x7x64 inputs -> 10 outputs
W3 = tf.get_variable("W3", shape=[4*4*128, 1024],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([1024]))

logits1 = tf.matmul(L2_flat, W3) + b
logits1=tf.nn.dropout(logits1,keep_prob=keep_prob)

W32 = tf.get_variable("W32", shape=[1024,500],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([500]))

logits2 = tf.matmul(logits1, W32) + b2
logits2=tf.nn.dropout(logits2,keep_prob=keep_prob)

W33 = tf.get_variable("W33", shape=[500,10],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(logits2, W33) + b3
logits=tf.nn.dropout(logits,keep_prob=keep_prob)


#fully connected network
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
saver = tf.train.Saver()

sess = tf.Session()# initialize
sess.run(tf.global_variables_initializer())


#saver=tf.train.import_meta_graph('./saver_data/my_test_model.meta')

#학습시간 측정
start_time = time.time()
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    if(epoch%3==0):
        saver.save(sess, './saver_data/my_test_model-%04d.ckpt' % (epoch))
        print("save ! epoch : ",epoch)
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys,keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')
#학습시간 출력
print("time elapsed: {:.2f}s".format(time.time() - start_time))
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels,keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1],keep_prob:1}))

# plt.imshow(mnist.test.images[r:r + 1].
#           reshape(28, 28), cmap='Greys', interpolation='nearest')
# plt.show()




C:\Users\Robotmedia9\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
save ! epoch :  0
Epoch: 0001 cost = 1.116491299
Epoch: 0002 cost = 0.549243058
Epoch: 0003 cost = 0.514467592
save ! epoch :  3
Epoch: 0004 cost = 0.498690496
Epoch: 0005 cost = 0.483960774
Epoch: 0006 cost = 0.483140360
save ! epoch :  6
Epoch: 0007 cost = 0.481128308
Epoch: 0008 cost = 0.475535799
Epoch: 0009 cost = 0.470567480
save ! epoch :  9
Epoch: 0010 cost = 0.469355920
Learning Finished!
time elapsed: 56.88s
Accuracy: 0.9896
Label:  [1]
Prediction:  [1]


In [4]:

saver = tf.train.Saver()
saver.restore(sess, "./saver_data/my_test_model-0000.ckpt")
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels,keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1],keep_prob:1}))


INFO:tensorflow:Restoring parameters from ./saver_data/my_test_model-0000.ckpt
Accuracy: 0.0892
Label:  [6]
Prediction:  [5]
